In [9]:
import segmentation_models as sm

sm.set_framework('tf.keras')

from cnn.input import PascalVOC2012Dataset, Dataloader, get_training_augmentation, get_preprocessing

import os

ImportError: cannot import name 'get_preprocessing' from 'cnn.input' (/home/guilherme/git/segqnas/cnn/input.py)

In [ ]:
train_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'train.txt',
)

val_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'val.txt',
)

images_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'JPEGImages'
)

masks_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'SegmentationClass'
) 

In [ ]:
image_height = 128
image_width = 128
backbone = 'efficientnetb0'

In [ ]:
train_dataset = PascalVOC2012Dataset(
    train_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
    augmentation=get_training_augmentation(image_height, image_width),
    preprocessing=get_preprocessing(sm.get_preprocessing(backbone))
)

val_dataset = PascalVOC2012Dataset(
    val_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
    preprocessing=get_preprocessing(sm.get_preprocessing(backbone))
)

In [ ]:
train_dataloader = Dataloader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = Dataloader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
net = sm.FPN(backbone, classes=21, input_shape=(image_height, image_width, 3), activation='softmax', encoder_weights='imagenet')

2022-07-10 14:40:51.321503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 14:40:51.321682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guilherme/git/segqnas/.venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-10 14:40:51.321736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guilherme/git/segqnas/.venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-10 14:40:51.321779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcub

In [ ]:
net.compile(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    optimizer='Adam',
    loss=sm.losses.categorical_focal_dice_loss,
    metrics=[sm.metrics.IOUScore(threshold=0.5)],
)

In [ ]:
history = net.fit(train_dataloader, 
                validation_data=val_dataloader,
                epochs=50)

Epoch 1/50
91/91 [==============================] - 343s 4s/step - loss: 0.8614 - iou_score: 0.2707 - val_loss: 0.9379 - val_iou_score: 0.1664
Epoch 2/50
91/91 [==============================] - 336s 4s/step - loss: 0.7786 - iou_score: 0.2399 - val_loss: 0.8471 - val_iou_score: 0.2025
Epoch 3/50
91/91 [==============================] - 331s 4s/step - loss: 0.7385 - iou_score: 0.2587 - val_loss: 0.8149 - val_iou_score: 0.2662
Epoch 4/50
91/91 [==============================] - ETA: 0s - loss: 0.7046 - iou_score: 0.2784

KeyboardInterrupt: 